# Visualize various metrics

This notebook provides an example of looking at different metrics to identify effects of different events (e.g. stay at home orders, outlier deaths, etc.)

Make sure to run batch model fitting to generate `data/metro_areas.csv` before running this notebook.

```
python fit_models.py --specfile=metro_areas
```

In [ ]:
import itertools
import numpy as np
import pandas as pd
from datetime import datetime, timezone, timedelta
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from scipy import optimize
import statsmodels.api as sm
import os
import pickle
import requests

from modeling import dataproc, sir_model

# Utility functions

In [ ]:
def plot_sir_model(r, i, total_model_days, df, metric, sampling_rate, name):
    """Plot the model death rates and total deaths vs actual data.
    
    Args:
        r: Array holding daily recovered population values from SIR model
        i: Array holding daily infected population values from SIR model
        total_model_days: Total number of modeled days to plot
        df: Dataframe holding metric values.
        metric: The type of metric to plot ('Cases' or 'Deaths')
        sampling_rate: Number of samples per day used to simulate the model.
        name: A name to attach to the plot.
    """
    plot_start_time = df['Date'].min().timestamp()
    plot_step_size = 24 * 60 * 60 / sampling_rate
    plot_end_time = plot_start_time + total_model_days * 24 * 60 * 60 
    plot_timestamps = np.arange(plot_start_time, plot_end_time, plot_step_size)
    plot_dates = [datetime.utcfromtimestamp(x) for x in plot_timestamps]
    print('peak date', plot_dates[np.argmax(i)])
    # Plot peak infection
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.ticklabel_format(useOffset=False)
    ax.ticklabel_format(style='plain')
    ax.plot(plot_dates[:-sampling_rate],
            (r[sampling_rate:] - r[:-sampling_rate]),
            c='g',
            label='model ' + metric + ' rate',
            linewidth=4)
    ax.plot(df['Date'].to_list()[:-1],
            (df[metric] - df[metric].shift())[1:], label='actual ' + metric + ' rate', c='r', linewidth=4)
    ax.set_title('SIR model for ' + name)
    ax.set_xlabel('Number of days')
    ax.set_ylabel('Number of individuals')
    plt.legend()
    plt.plot()
    
    # Plot recovery
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.ticklabel_format(useOffset=False)
    ax.ticklabel_format(style='plain')
    ax.plot(plot_dates, r, c='g',
            label='model ' + metric, linewidth=4)
    ax.plot(df['Date'].to_list(), df[metric], label='actual ' + metric, c='r', linewidth=4)
    ax.set_title('SIR model for ' + name)
    ax.set_xlabel('Number of days')
    ax.set_ylabel('Number of individuals')
    plt.legend()
    plt.show()

# Load Covid-19 and Census Data

In [ ]:
datastore = dataproc.DataStore()

# Load model params dataframe

In [ ]:
model_df = pd.read_csv('data/metro_areas.csv', index_col='Unnamed: 0', parse_dates=['Date'])
model_df

## Sheltering in place effect on infection rate

How many people does an infected person infect per day? Models trained with new data may reveal a sudden change in this parameter based on sheltering-in-place orders. For example, New York, Michigan, and Louisiana all implement sheltering-in-place orders around 3/22-3/24, and the "infection rate" based on deaths suddenly dropped about a week later.

In [ ]:
plt.figure(figsize=(10, 8))

shelter_dates_7_days = {'NYC': '2020-03-29',
                        'Detroit': '2020-03-31'
                       }

for area in ['NYC', 'Detroit', 'New Orleans']:
    model_area_df = model_df[model_df['Area'] == area]
    plt.plot(model_area_df['Date'], model_area_df['Infection Rate'], linewidth=5, label=area)
plt.legend()

## Anomaly detection

Can we use the model MSE to detect anomalies in the death rate?

Notice that there seems to be a recent sudden shift in prediction error around 4/15 for New Orleans, 4/16 for Detroit and New York! Why?

In [ ]:
for area in ['NYC', 'Detroit', 'New Orleans']:
    model_area_df = model_df[(model_df['Area'] == area) & (model_df['Date'] <= '2020-04-18')]
    plt.plot(model_area_df['Date'], model_area_df['MSE'], linewidth=5, label=area)
    plt.legend()
    plt.show()


In [ ]:
# Check data for one such area and date range
model_df[(model_df['Area'] == 'New Orleans') & (model_df['Date'] >= '2020-04-12') & (model_df['Date'] <= '2020-04-18')]

In [ ]:
# Use a past date to build a model and see what happened on anomalous days

AREA_OF_INTEREST = ('US', 'NYC', 36, [5, 47, 61, 81, 85])
#AREA_OF_INTEREST = ('US', 'New Orleans', 22, [51, 71, 75, 87, 89, 95, 103, 105])
#AREA_OF_INTEREST = ('US', 'Detroit', 26, [87, 93, 99, 125, 147, 163])

MODEL_FIT_FIRST_DATE = '2019-01-01'
MODEL_FIT_LAST_DATE = '2020-04-14'  # Fit model to data before this date, reserving later dates as holdout.
METRIC = 'Deaths'
SAMPLING_RATE = 10
SIMULATION_DAYS = 90

In [ ]:
if len(AREA_OF_INTEREST) <= 2:
    area_df, population = datastore.get_time_series_for_area(AREA_OF_INTEREST[0])
else:
    area_df, population = datastore.get_time_series_for_area(
        AREA_OF_INTEREST[0], AREA_OF_INTEREST[2], AREA_OF_INTEREST[3])

#Validation plot
validation_area_df = area_df # TODO: add holdout days
validation_area_df = validation_area_df[(validation_area_df.Date >= MODEL_FIT_FIRST_DATE)]
validation_area_df = validation_area_df[validation_area_df[METRIC] > 0]
validation_area_df = validation_area_df.sort_values(by=['Date']).reset_index(drop=True)

# best_pop_frac = best_param[0]
# best_infection_rate = best_param[1]
# best_multiplier = best_param[2]
model_row = model_df[(model_df['Date'] == MODEL_FIT_LAST_DATE) & (model_df['Area'] == AREA_OF_INTEREST[1])]
best_pop_frac = model_row.iloc[0]['Pop Deaths Rate']
best_infection_rate = model_row.iloc[0]['Infection Rate']
best_multiplier = model_row.iloc[0]['Deaths Multiplier']
days_to_recover = model_row.iloc[0]['Days To Recover']


infected = validation_area_df[METRIC].iloc[0] * best_multiplier
t, s, i, r = sir_model.compute_sir(
    SAMPLING_RATE,
    SIMULATION_DAYS,
    population * best_pop_frac,
    infected,
    best_infection_rate,
    days_to_recover
)

valid_obj = sir_model.create_objective_fn(
    validation_area_df[METRIC].to_numpy(), population, sampling_rate=SAMPLING_RATE)
validation_mse = valid_obj(best_pop_frac, best_infection_rate, days_to_recover, best_multiplier)

print('Steady state population fraction affected:', best_pop_frac)
print('Steady state number affected (e.g. total expected deaths):', best_pop_frac * population)
print('Transmissions per person per day:', best_infection_rate)
print('First day estimate multiplier', best_multiplier)
print('Validation MSE', validation_mse)
plot_sir_model(r, i, SIMULATION_DAYS, validation_area_df, METRIC, SAMPLING_RATE, AREA_OF_INTEREST[1])

## Example reports

Doing a news search for these regions: Seems NYC is reporting the [new record number of deaths](https://www.nytimes.com/2020/04/18/nyregion/coronavirus-deaths-nyc.html), but the anomalous data is due to probably bad daily reporting (double on 4/17, zero on 4/18).